# This notebook is designed for teaching/testing purposes to help you visualize the tensor shapes that go through each module. Read it side-by-side with 'model.py'

In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, './venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

In [2]:
# tokenizer
from tokenizer import get_tokenizer
tokenizer = get_tokenizer(size = 1024) # size options are 95, 128, 256, 512, 1024 and 2048

# config file
from model_code.baseGPT_config import ModelConfig
cfg = ModelConfig()
cfg.vocab_len = tokenizer.vocab_len
print(cfg)

import random
import torch

ModelConfig(dim=64, vocab_len=1027, device='cpu', num_layers=10, second_resid_norm=False, mlp_hidden_mult=1, mlp_bias=False, mlp_nonlinearity='SiLU', mlp_gated=True, num_q_heads=10, num_kv_heads=2, head_dim=16, theta=10000, max_seq_len=512, scale_first_resid=True, norm_type='RMSNorm', norm_affine=True, norm_bias=True, eps=1e-06, max_batch_size=1)


# Norms

In [3]:
### RMSNorm
from model_code.modules.norms import Norm

# Create an instance of RMSNorm
hold = cfg.norm_type
cfg.norm_type = 'rmsnorm' # purposely mis-typing it
module = Norm(cfg.dim, cfg)

# let's take a look
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

# Initially, logging is disabled by default
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('CosineNorm')
#module.disable_function_logging('LayerNorm')
#module.disable_function_logging('RMSNorm')

x = torch.randn(32,cfg.max_seq_len,cfg.dim)

# Call the forward method - logging will occur
output = module(x)

# Disable logging. 
# This isn't actually necessary since we won't be using this object again but that's how you'd do it
module.disable_logging()

# clearing up ram jic we're training later
cfg.norm_type = hold
del hold, module, x, output

norm type rmsnorm not found. defaulting to RMSNorm
0.128 K parameters
Norm()

==========Entering Norm.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])

==========Entering Norm.RMSNorm==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 64])
==========Exiting Norm.RMSNorm==========

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 64])
==========Exiting Norm.forward==========


In [4]:
# LayerNorm
hold = cfg.norm_type
cfg.norm_type = 'LayerNorm'
module = Norm(cfg.dim, cfg)
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('CosineNorm')
#module.disable_function_logging('LayerNorm')
#module.disable_function_logging('RMSNorm')

x = torch.randn(32,cfg.max_seq_len,cfg.dim)
output = module(x)
module.disable_logging()
cfg.norm_type = hold
del hold, module, x, output


==========Entering Norm.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])

==========Entering Norm.LayerNorm==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 64])
==========Exiting Norm.LayerNorm==========

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 64])
==========Exiting Norm.forward==========


In [5]:
# CosineNorm
hold = cfg.norm_type
cfg.norm_type = 'CosineNorm'
module = Norm(cfg.dim, cfg)
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('CosineNorm')
#module.disable_function_logging('LayerNorm')
#module.disable_function_logging('RMSNorm')

x = torch.randn(32,cfg.max_seq_len,cfg.dim)
output = module(x)
module.disable_logging()
cfg.norm_type = hold
del hold, module, x, output


==========Entering Norm.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])

==========Entering Norm.CosineNorm==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 64])
==========Exiting Norm.CosineNorm==========

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 64])
==========Exiting Norm.forward==========


# Attention

In [6]:
# first up let's look at training
from model_code.modules.attentions import precompute_freqs_cis, MQSA

# Create an instance of multi-head self-attention
module = MQSA(cfg)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

# Initially, logging is disabled by default
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('apply_rotary_emb')
#module.disable_function_logging('reshape_for_broadcast')
#module.disable_function_logging('match_headcount')
#module.disable_function_logging('attend')
#module.disable_function_logging('calc_output')

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.head_dim,
    cfg.max_seq_len,
    cfg.theta
)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)
x = torch.randn(32,cfg.max_seq_len,cfg.dim)

# Call the forward method - logging will occur
output = module(x, freqs_cis, mask, training=True)

# Disable logging. 
# This isn't actually necessary since we won't be using this object again but that's how you'd do it
module.disable_logging()

# clearing up ram jic we're training later
del module, freqs_cis, mask, x, output

24.576 K parameters
MQSA(
  (Wq): Linear(in_features=64, out_features=160, bias=False)
  (Wk): Linear(in_features=64, out_features=32, bias=False)
  (Wv): Linear(in_features=64, out_features=32, bias=False)
  (Wo): Linear(in_features=160, out_features=64, bias=False)
)

==========Entering MQSA.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])
Tensor 'freqs_cis' shape: torch.Size([512, 8])
Tensor 'mask' shape: torch.Size([512, 512])
Integer 'cache_len': Value=True

==========Entering MQSA.apply_rotary_emb==========
Inputs:
Tensor 'xq' shape: torch.Size([32, 512, 10, 16])
Tensor 'xk' shape: torch.Size([32, 512, 2, 16])
Tensor 'freqs_cis' shape: torch.Size([512, 8])

==========Entering MQSA.reshape_for_broadcast==========
Inputs:
Tensor 'freqs_cis' shape: torch.Size([512, 8])
Tensor 'x' shape: torch.Size([32, 512, 10, 8])

Outputs:
Tensor 'output' shape: torch.Size([1, 512, 1, 8])
==========Exiting MQSA.reshape_for_broadcast==========

Outputs:
Tensor 'output[0]' shape

In [7]:
# now let's do it for inference

module = MQSA(cfg)
module.enable_logging()
#module.disable_function_logging('apply_rotary_emb')
#module.disable_function_logging('reshape_for_broadcast')
#module.disable_function_logging('match_headcount')
#module.disable_function_logging('attend')
#module.disable_function_logging('calc_output')

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.head_dim,
    cfg.max_seq_len,
    cfg.theta
)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)

# setting up for kv caching
context_chunk_len = cfg.max_seq_len // 4
cache_len = random.randint(1, 3 * context_chunk_len)
seq_len = cache_len + context_chunk_len
# need to extend the mask with zeros for the cached values
mask = mask[:context_chunk_len, :context_chunk_len]
mask = torch.hstack(
            [torch.zeros((context_chunk_len, cache_len)), mask]
        )

# these don't use seq_len because those entries should already be in the kv cache
freqs_cis = freqs_cis[:context_chunk_len]
x = torch.randn(cfg.max_batch_size,context_chunk_len,cfg.dim)

# Call the forward method - logging will occur
output = module(x, freqs_cis, mask, cache_len)

# Disable logging. 
# This isn't actually necessary since we won't be using this object again but that's how you'd do it
module.disable_logging()

# clearing up ram jic we're training later
del module, freqs_cis, mask, cache_len, context_chunk_len, seq_len, x, output


==========Entering MQSA.forward==========
Inputs:
Tensor 'x' shape: torch.Size([1, 128, 64])
Tensor 'freqs_cis' shape: torch.Size([128, 8])
Tensor 'mask' shape: torch.Size([128, 479])
Integer 'cache_len': Value=351

==========Entering MQSA.apply_rotary_emb==========
Inputs:
Tensor 'xq' shape: torch.Size([1, 128, 10, 16])
Tensor 'xk' shape: torch.Size([1, 128, 2, 16])
Tensor 'freqs_cis' shape: torch.Size([128, 8])

==========Entering MQSA.reshape_for_broadcast==========
Inputs:
Tensor 'freqs_cis' shape: torch.Size([128, 8])
Tensor 'x' shape: torch.Size([1, 128, 10, 8])

Outputs:
Tensor 'output' shape: torch.Size([1, 128, 1, 8])
==========Exiting MQSA.reshape_for_broadcast==========

Outputs:
Tensor 'output[0]' shape: torch.Size([1, 128, 10, 16])
Tensor 'output[1]' shape: torch.Size([1, 128, 2, 16])
==========Exiting MQSA.apply_rotary_emb==========

==========Entering MQSA.match_headcount==========
Inputs:
Tensor 'keys' shape: torch.Size([1, 479, 2, 16])
Tensor 'values' shape: torch.Siz

# MLP

In [8]:
from model_code.modules.feedforward import MLP

# GeGLU
module = MLP(
    cfg.dim, 
    cfg.dim * cfg.mlp_hidden_mult, 
    cfg.dim, 
    'GeLU', 
    gated=True, 
    bias=False, 
    dropout_rate = 0.1
)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)
module.enable_logging()

x = torch.randn(32,cfg.max_seq_len,cfg.dim)
output = module(x, training=True)
module.disable_logging()
del module, x, output

12.288 K parameters
MLP(
  (Wgate): Linear(in_features=64, out_features=64, bias=False)
  (Wup): Linear(in_features=64, out_features=64, bias=False)
  (Wdown): Linear(in_features=64, out_features=64, bias=False)
  (nonlinearity): GELU(approximate='none')
)

==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])
Integer 'training': Value=True

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 64])
==========Exiting MLP.forward==========


In [9]:
# not gated, testing every other nonlinearity
module = MLP(
    cfg.dim, 
    cfg.dim * cfg.mlp_hidden_mult, 
    cfg.dim, 
    'ReLU', 
    gated=False, 
    bias=False, 
    dropout_rate = 0.1
)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)
module.enable_logging()

x = torch.randn(32,cfg.max_seq_len,cfg.dim)
output = module(x, training=True)
module.disable_logging()
del module, x, output

8.192 K parameters
MLP(
  (Wup): Linear(in_features=64, out_features=64, bias=False)
  (Wdown): Linear(in_features=64, out_features=64, bias=False)
  (nonlinearity): ReLU()
)

==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])
Integer 'training': Value=True

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 64])
==========Exiting MLP.forward==========


# ResidualLayer

In [10]:
from model_code.baseGPT import ResidualLayer

# TRAINING
module = ResidualLayer(cfg)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
#module.disable_function_logging('attn_connect')
#module.disable_function_logging('mlp_connect')
### enabling printing for sub-modules
#module.pre_attn_norm.enable_logging()
#module.attn.enable_logging()
#module.post_attn_norm.enable_logging()
#module.pre_mlp_norm.enable_logging()
#module.mlp.enable_logging()
#module.post_mlp_norm.enable_logging()

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.head_dim,
    cfg.max_seq_len,
    cfg.theta
)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)
x = torch.randn(32,cfg.max_seq_len,cfg.dim)

output = module(x, freqs_cis, mask, training=True)
module.disable_logging()
del module, x, output

37.12 K parameters
ResidualLayer(
  (pre_attn_norm): Norm()
  (attn): MQSA(
    (Wq): Linear(in_features=64, out_features=160, bias=False)
    (Wk): Linear(in_features=64, out_features=32, bias=False)
    (Wv): Linear(in_features=64, out_features=32, bias=False)
    (Wo): Linear(in_features=160, out_features=64, bias=False)
  )
  (pre_mlp_norm): Norm()
  (mlp): MLP(
    (Wgate): Linear(in_features=64, out_features=64, bias=False)
    (Wup): Linear(in_features=64, out_features=64, bias=False)
    (Wdown): Linear(in_features=64, out_features=64, bias=False)
    (nonlinearity): SiLU()
  )
)

==========Entering ResidualLayer.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])
Tensor 'freqs_cis' shape: torch.Size([512, 8])
Tensor 'mask' shape: torch.Size([512, 512])
Integer 'cache_len': Value=True

==========Entering ResidualLayer.attn_connect==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 64])
Tensor 'freqs_cis' shape: torch.Size([512, 8])
Tensor 'mask' shape: t

In [11]:
# INFERENCE
module = ResidualLayer(cfg)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
#module.disable_function_logging('attn_connect')
#module.disable_function_logging('mlp_connect')
#module.pre_attn_norm.enable_logging()
#module.attn.enable_logging()
#module.post_attn_norm.enable_logging()
#module.pre_mlp_norm.enable_logging()
#module.mlp.enable_logging()
#module.post_mlp_norm.enable_logging()

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.head_dim,
    cfg.max_seq_len,
    cfg.theta
)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)
# setting up for kv caching
cache_len = 420
context_chunk_len = 64
seq_len = cache_len + context_chunk_len
# need to extend the mask with zeros for the cached values
mask = mask[:context_chunk_len, :context_chunk_len]
mask = torch.hstack(
            [torch.zeros((context_chunk_len, cache_len)), mask]
        )
# these don't use seq_len because those entries should already be in the kv cache
freqs_cis = freqs_cis[:context_chunk_len]
x = torch.randn(1,context_chunk_len,cfg.dim)

output = module(x, freqs_cis, mask, cache_len)
module.disable_logging()
del module, freqs_cis, mask, cache_len, context_chunk_len, seq_len, x, output

37.12 K parameters
ResidualLayer(
  (pre_attn_norm): Norm()
  (attn): MQSA(
    (Wq): Linear(in_features=64, out_features=160, bias=False)
    (Wk): Linear(in_features=64, out_features=32, bias=False)
    (Wv): Linear(in_features=64, out_features=32, bias=False)
    (Wo): Linear(in_features=160, out_features=64, bias=False)
  )
  (pre_mlp_norm): Norm()
  (mlp): MLP(
    (Wgate): Linear(in_features=64, out_features=64, bias=False)
    (Wup): Linear(in_features=64, out_features=64, bias=False)
    (Wdown): Linear(in_features=64, out_features=64, bias=False)
    (nonlinearity): SiLU()
  )
)

==========Entering ResidualLayer.forward==========
Inputs:
Tensor 'x' shape: torch.Size([1, 64, 64])
Tensor 'freqs_cis' shape: torch.Size([64, 8])
Tensor 'mask' shape: torch.Size([64, 484])
Integer 'cache_len': Value=420

==========Entering ResidualLayer.attn_connect==========
Inputs:
Tensor 'x' shape: torch.Size([1, 64, 64])
Tensor 'freqs_cis' shape: torch.Size([64, 8])
Tensor 'mask' shape: torch.Siz

# Full Model

In [12]:
from model_code.baseGPT import baseGPT

# TRAINING
module = baseGPT(cfg)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
### enabling printing for sub-modules
module.layers[0].enable_logging()
module.final_norm.enable_logging()

input_token_ids = torch.randint(tokenizer.vocab_len, (32, cfg.max_seq_len))
target_token_ids = torch.randint(tokenizer.vocab_len, (32, cfg.max_seq_len))

output, loss = module(input_token_ids, target_token_ids=target_token_ids)
print(loss)
module.disable_logging()
del module, input_token_ids, target_token_ids, output

437.056 K parameters
baseGPT(
  (token_embedder): Embedding(1027, 64)
  (layers): ModuleList(
    (0-9): 10 x ResidualLayer(
      (pre_attn_norm): Norm()
      (attn): MQSA(
        (Wq): Linear(in_features=64, out_features=160, bias=False)
        (Wk): Linear(in_features=64, out_features=32, bias=False)
        (Wv): Linear(in_features=64, out_features=32, bias=False)
        (Wo): Linear(in_features=160, out_features=64, bias=False)
      )
      (pre_mlp_norm): Norm()
      (mlp): MLP(
        (Wgate): Linear(in_features=64, out_features=64, bias=False)
        (Wup): Linear(in_features=64, out_features=64, bias=False)
        (Wdown): Linear(in_features=64, out_features=64, bias=False)
        (nonlinearity): SiLU()
      )
    )
  )
  (final_norm): Norm()
  (criterion): CrossEntropyLoss()
)

==========Entering baseGPT.forward==========
Inputs:
Tensor 'input_token_ids' shape: torch.Size([32, 512])
Tensor 'cache_len' shape: torch.Size([32, 512])

==========Entering ResidualLayer.f

In [13]:
# Inference
module = baseGPT(cfg)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
### enabling printing for sub-modules
#for i in range(cfg.num_layers):
    #module.layers[i].enable_logging()
#module.final_norm.enable_logging()

input_token_ids = torch.randint(tokenizer.vocab_len, (1, cfg.max_seq_len // 4))

output, _ = module(input_token_ids, cache_len = cfg.max_seq_len // 3)
print(output.shape)
module.disable_logging()
del module, input_token_ids, output

437.056 K parameters
baseGPT(
  (token_embedder): Embedding(1027, 64)
  (layers): ModuleList(
    (0-9): 10 x ResidualLayer(
      (pre_attn_norm): Norm()
      (attn): MQSA(
        (Wq): Linear(in_features=64, out_features=160, bias=False)
        (Wk): Linear(in_features=64, out_features=32, bias=False)
        (Wv): Linear(in_features=64, out_features=32, bias=False)
        (Wo): Linear(in_features=160, out_features=64, bias=False)
      )
      (pre_mlp_norm): Norm()
      (mlp): MLP(
        (Wgate): Linear(in_features=64, out_features=64, bias=False)
        (Wup): Linear(in_features=64, out_features=64, bias=False)
        (Wdown): Linear(in_features=64, out_features=64, bias=False)
        (nonlinearity): SiLU()
      )
    )
  )
  (final_norm): Norm()
  (criterion): CrossEntropyLoss()
)

==========Entering baseGPT.forward==========
Inputs:
Tensor 'input_token_ids' shape: torch.Size([1, 128])
Integer 'cache_len': Value=170

Outputs:
Tensor 'output[0]' shape: torch.Size([1, 12